In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import silhouette_score
import pandas as pd
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Seleccionar las columnas relevantes para el clustering jerárquico
X = df_filtrado[['Recencia', 'Monto', 'Frecuencia']]

# Crear lista de métodos de enlace
linkage_methods = ['ward', 'complete', 'average', 'single']

# Configurar la visualización
plt.figure(figsize=(12, 8))

# Iterar sobre los métodos de enlace y realizar clustering jerárquico
for i, method in enumerate(linkage_methods):
    # Realizar clustering jerárquico con el método de enlace actual
    clustering = AgglomerativeClustering(n_clusters=3, linkage=method)
    labels = clustering.fit_predict(X)
    sc = silhouette_score(X, labels)

    # Graficar los puntos en un diagrama de dispersión para las dos primeras columnas
    plt.subplot(2, 2, i+1)
    plt.scatter(X['Recencia'], X['Monto'], c=labels, cmap='viridis')
    plt.xlabel('Recencia')
    plt.ylabel('Monto')
    plt.title(f'Clustering jerárquico - Enlace: {method} - s-score: {sc:.2f}')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer

# Estandarizar los datos
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_filtrado[['Recencia', 'Frecuencia', 'Monto']])

In [ ]:


# Definir una lista de posibles valores de k para el método del codo
k_values = range(2, 8)

# Inicializar listas para almacenar las métricas
inertia_values = []

# Realizar clustering con diferentes valores de k y calcular las métricas
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    labels = kmeans.labels_
    # Calcular la inercia
    inertia_values.append(kmeans.inertia_)

# Graficar el método del codo utilizando la inercia
plt.plot(k_values, inertia_values, 'bo-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo')

plt.show()

In [ ]:
# Aplicar algoritmos de clustering (por ejemplo, KMeans)
kmeans = KMeans(n_clusters=3, random_state=42)
df_filtrado['Cluster'] = kmeans.fit_predict(df_scaled)

# Visualizar los resultados
plt.scatter(df_filtrado['Recencia'], df_filtrado['Monto'], c=df_filtrado['Cluster'], cmap='viridis', alpha=0.7)
plt.xlabel('Recencia')
plt.ylabel('Monto')
plt.title('Clusters generados por KMeans')

# Obtener etiquetas únicas de los clusters
unique_labels = df_filtrado['Cluster'].unique()

# Generar etiquetas para la leyenda
legend_labels = [f'Cluster {label}' for label in unique_labels]
print(legend_labels)
# Agregar leyenda
plt.legend(legend_labels, loc='upper right')
plt.show()


In [ ]:

pca = PCA(n_components=2)  # Crear el modelo PCA
df_pca = pca.fit_transform(df_scaled)  # Transformar los datos

# Opcional: Para acceder a la varianza explicada
explained_variance = pca.explained_variance_ratio_


In [ ]:
explained_variance

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=["PC1", "PC2"])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear un scatter plot de las dos primeras componentes principales
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_pca.iloc[:, 0], y=df_pca.iloc[:, 1], hue=df_filtrado['Cluster'], palette="coolwarm")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.title("Visualización de PCA en 2D")
plt.show()



In [ ]:
score = silhouette_score(df_pca, df_filtrado['Cluster'])
print("Silhouette Score:", score)

In [ ]:

# Crear el modelo DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Ajusta eps y min_samples según sea necesario
df_filtrado["Cluster_DBSCAN"] = dbscan.fit_predict(df_pca)

# Visualizar los resultados
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_pca.iloc[:, 0], y=df_pca.iloc[:, 1], hue=df_filtrado["Cluster_DBSCAN"], palette="tab10", legend="full")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.title("DBSCAN aplicado a los datos reducidos con PCA")
plt.legend(title="Cluster (DBSCAN)")
plt.show()


In [ ]:
score = silhouette_score(df_pca, df_filtrado['Cluster_DBSCAN'])
print("Silhouette Score:", score)

In [ ]:


# Definir rangos para eps y min_samples
eps_range = np.arange(0.1, 1.0, 0.1)  # Valores de eps
min_samples_range = range(3, 11)      # Valores de min_samples

# Variables para almacenar los mejores parámetros
best_score = -1
best_params = {"eps": None, "min_samples": None}

# GridSearch manual
for eps in eps_range:
    for min_samples in min_samples_range:
        # Aplicar DBSCAN
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(df_pca)
        
        # Ignorar configuraciones con un único clúster o todos ruido
        if len(set(labels)) <= 1 or (labels == -1).mean() == 1.0:
            continue
        
        # Calcular Silhouette Score
        score = silhouette_score(df_pca, labels)
        
        # Guardar si es el mejor hasta ahora
        if score > best_score:
            best_score = score
            best_params = {"eps": eps, "min_samples": min_samples}

# Resultados
print("Mejor Silhouette Score:", best_score)x
print("Mejores parámetros:", best_params)

In [ ]:

df_filtrado['CLUSTER_BEST_INT'] = df_filtrado['Cluster_DBSCAN_BEST'].astype(int)
# Identificar columnas categóricas
categorical_cols = df_filtrado.select_dtypes(include=['category', 'object']).columns

# Excluir columnas categóricas antes de calcular el promedio
numerical_cols = df_filtrado.drop(columns=categorical_cols).columns

# Agrupación por "CLUSTER_BEST_INT" y cálculo de la media
cluster_summary = df_filtrado[numerical_cols].groupby(df_filtrado["CLUSTER_BEST_INT"]).mean()
cluster_summary
